In [5]:
import re 
import csv
import json
import nltk
import sklearn
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup 
import matplotlib.pyplot as plt
pd.set_option("display.max_colwidth",-1)


def rescue_code(function):
    import inspect
    get_ipython().set_next_input("".join(inspect.getsourcelines(function)[0]))
    

    
def tic():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print("Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
        print("Toc: start time not set")
        
        
        
with open('drama_love_reviews_100.json') as json_data:
    posReviewsDrama = json.load(json_data)
    
with open('drama_hate_reviews_100.json') as json_data:
    negReviewsDrama = json.load(json_data)
    
with open('action_love_reviews_100.json') as json_data:
    posReviewsAction = json.load(json_data)
    
with open('action_hate_reviews_100.json') as json_data:
    negReviewsAction  = json.load(json_data)
    
with open('horror_hate_reviews_100.json') as json_data:
    negReviewsHorror  = json.load(json_data) 
    
with open('horror_love_reviews_100.json') as json_data:
    posReviewsHorror  = json.load(json_data) 
    
with open('romance_hate_reviews_100.json') as json_data:
    negReviewsRomance  = json.load(json_data)     
    
with open('romance_love_reviews_100.json') as json_data:
    posReviewsRomance  = json.load(json_data) 
    
with open('Sci-Fi_love_reviews_100.json') as json_data:
    posReviewsSci  = json.load(json_data) 
    
with open('Sci-Fi_hate_reviews_100.json') as json_data:
    negReviewsSci  = json.load(json_data) 

In [6]:
posReList = []
for reviewlist in posReviewsDrama + posReviewsHorror + posReviewsAction + posReviewsSci + posReviewsRomance:
    if reviewlist is not None:
        for review in reviewlist:
            posReList.append([review['review'],review['categorie']])
            
posReviews = pd.DataFrame(posReList,columns=['reviews', 'sentiment'])
posReviews = posReviews.drop_duplicates()

negReList = []
for reviewlist in  negReviewsAction + negReviewsHorror +negReviewsDrama +negReviewsRomance + negReviewsSci:
    if reviewlist is not None:
        for review in reviewlist:
            negReList.append([review['review'],review['categorie']])
            
negReviews = pd.DataFrame(negReList,columns=['reviews', 'sentiment'])
negReviews = negReviews.drop_duplicates()

In [7]:
from sklearn.model_selection import train_test_split

pos_train, pos_test = train_test_split(posReviews, test_size = 0.3)
neg_train, neg_test = train_test_split(negReviews, test_size = 0.3)

train = pos_train.append(neg_train, ignore_index=True)
test = pos_test.append(neg_test, ignore_index=True)

train = train.reset_index(drop=True)
train['id'] = train.index


test = test.reset_index(drop=True)
test['id'] = test.index

print(train.shape)
print(test.shape)


(34079, 3)
(14606, 3)


In [8]:
#train['reviews'] = train['reviews'].apply(lambda x : x.lower())
#test['reviews'] = test['reviews'].apply(lambda x : x.lower())
#re.sub("[^a-zA-Z]",' ',train.ix[0,'reviews'])

def textPrepocess(movieReviews):
    movieReviews = movieReviews.lower()
    movieReviews = re.sub("[^a-zA-Z]",' ',movieReviews)
    movieReviews = (" ".join(movieReviews.split()))
    return movieReviews

train['reviews'] = train['reviews'].apply(textPrepocess)
test['reviews'] = test['reviews'].apply(textPrepocess)

#### Word2vec
Word2vec is a **neural network implementation** that learns **distributed representations** for words. Other deep or recurrent neural network architectures had been proposed for learning word representations prior to this, but the major problem with these was the long time required to train the models. Word2vec learns quickly relative to other models.

1. Titles :             no titles
2. Features Extraction: CountVectorizer()
3. Stopwords :          Not remove
4. Classifier :         Randomforest
5. N-gram:              (1,1)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word" ,min_df=1)
train_features = vectorizer.fit_transform(train['reviews'])
train_features = train_features.toarray()
vocabulary = vectorizer.get_feature_names()

tic()
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest_100 = RandomForestClassifier(n_estimators = 100) 

test_features = vectorizer.transform(test['reviews'])
test_features = test_features.toarray()

# Use the random forest to make sentiment label predictions
# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest_100 = forest_100.fit(train_features, train["sentiment"])
result = forest_100.predict(test_features)
toc()

from sklearn.metrics import classification_report

target_names = ['postive', 'negative']
print(classification_report(test['sentiment'], result, target_names=target_names))


Elapsed time is 936.9531109333038 seconds.
             precision    recall  f1-score   support

    postive       0.80      0.69      0.74      6860
   negative       0.75      0.85      0.80      7746

avg / total       0.78      0.77      0.77     14606

